In [ ]:
# The following is the SVM implimentation for the SVM
# Files needed: file of negative questions and posistive questions (new_neg_data_3.txt, new_pos_data_3.txt)
# Dependecy tree features imported from pickle file created separately
# This file must have all glove dependencies imported in order to run

In [1]:
#SVM Precision recall and F score (after tuning)
vsmdata_home = "vsmdata"
glove_home = "glove.6B"

import os
import sys
import unicodecsv as csv
import random
import itertools
from operator import itemgetter
from collections import defaultdict
import numpy as np
import scipy
import scipy.spatial.distance
from numpy.linalg import svd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import utils
import numpy
from sklearn.metrics import classification_report, accuracy_score, f1_score

glv = utils.build_glove(os.path.join(glove_home, 'glove.6B.50d.txt'))

def cosine(u, v):        
    """Cosine distance between 1d np.arrays `u` and `v`, which must have 
    the same dimensionality. Returns a float."""
    # Use scipy's method:
    return scipy.spatial.distance.cosine(u, v)
    # Or define it yourself:
    # return 1.0 - (np.dot(u, v) / (vector_length(u) * vector_length(v)))

#neighbors(word='gnarly', mat=gnmat_lsa[0], rownames=gnmat_lsa[1])


def distance_between_words(word1, word2, mat = glv[0], rownames=glv[1], distfunc=cosine):
    if word1 not in rownames or word2 not in rownames:
        raise ValueError('%s is not in this VSM' % word1)
    w1 = mat[rownames.index(word1)]
    w2 = mat[rownames.index(word2)]
    
    return distfunc(w1,w2)
    
def distance_between_sentences(str1, str2, mat = glv[0], rownames = glv[1], distfunc = cosine):
    str1_list = str1.split()
    str2_list = str2.split()
    
    str1_vecs = [mat[rownames.index(word)] for word in str1_list]
    str2_vecs = [mat[rownames.index(word)] for word in str2_list]
    
    str1_combined_vecs = 0
    str2_combined_vecs = 0
    
    for vec in str1_vecs:
        str1_combined_vecs += vec
    
    for vec in str2_vecs:
        str2_combined_vecs += vec
        
    return distfunc(str1_combined_vecs, str2_combined_vecs)

In [2]:
import gensim #make sure you did pip install gensim
import gensim.models.doc2vec as doc2vec
from gensim.models import Doc2Vec

def make_distance_vectors(file_name):
    #expects a file with one sentence per line
    #Returns a dict with format {sentence : vector}
    
    assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be sloww otherwise"

    data_file = "data.txt" 

    with open(file_name) as f:
        file_content = f.readlines()
    
    
    sentences = doc2vec.TaggedLineDocument(file_name)
    model = Doc2Vec(sentences, size=100, window=8, min_count=5, workers=4, iter=20) #can take a while, can also modify parameters to change vectors
    model.save("doc2vec_model_1")

    model = Doc2Vec.load("doc2vec_model_1") #unnecessary in this case, but can use it after you have already created a model
    vector_dict = []

    cur_pair = 0
    for i in range(0, len(file_content), 2):
        arr = cosine(model.docvecs[i], model.docvecs[i+1])
        vector_dict.append(arr)
        #arr.append(distance_between_sentences(file_content[i],file_content[i+1]))
        #vector_dict.append([distance_between_sentences(file_content[i],file_content[i+1]), cosine(model.docvecs[i], model.docvecs[i+1])])
        #vector_dict.append(distance_between_sentences(model.docvecs[i], model.docvecs[i+1]))
        cur_pair += 1
    #print cur_pair
    return (vector_dict, file_content)

In [3]:
import sklearn
from sklearn import svm

def train(X, y,c, g):
    clf = svm.SVC(C = c, gamma = g)
    clf.fit(X,y)
    print 'finished training svm'
    return clf

def test(clf, X):
    predictions = clf.predict(X)
    print 'finished predicting'
    return predictions

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

In [4]:
def word_overlap(s1, s2):
    overlap = 0
    s1_tokenize = s1.split()
    s2_tokenize = s2.split()
    used_array = []
    for token in s1_tokenize:
        if token in s2_tokenize:
            if token not in used_array:
                overlap += 1
                used_array.append(token)
    return overlap

print word_overlap("hi hi", "hi hi") 

1


In [5]:
def jaccard(sentence1, sentence2):
    s1_tokenize = sentence1.split()
    s2_tokenize = sentence2.split()
    s1_set = set(s1_tokenize)
    s2_set = set(s2_tokenize)
    #print s1_set.intersection(s2_set)
    #print s1_set.union(s2_set)
    return float(len(s1_set.intersection(s2_set)))/float(len(s1_set.union(s2_set)))

In [6]:
def euclidean(u, v):    
    """Eculidean distance between 1d np.arrays `u` and `v`, which must 
    have the same dimensionality. Returns a float."""
    # Use scipy's method:
    return scipy.spatial.distance.euclidean(u, v)
    # Or define it yourself:
    # return vector_length(u - v) 

In [7]:
question_words = ["who", "what", "when", "where", "why", "how"]
def qword_indicator(s1, s2):
    s1_tokenize = s1.split()
    s2_tokenize = s2.split()
    s1_set = set(s1_tokenize)
    s2_set = set(s2_tokenize)
    for word in question_words:
        if word in s1_set.intersection(s2_set):
            return 1
        
    return 0

In [8]:
def bigram_overlap(s1, s2):
    s1_tokenize = s1.split()
    s2_tokenize = s2.split()
    bigram_overlap = 0
    s1_bigrams = set([(s1_tokenize[i], s1_tokenize[i+1]) for i in range(0, len(s1_tokenize)-1)])
    s2_bigrams = set([(s2_tokenize[i], s2_tokenize[i+1]) for i in range(0, len(s2_tokenize)-1)])
    bigram_overlap = len(s1_bigrams.intersection(s2_bigrams))
    return bigram_overlap

bigram_overlap("How are you doing", "How are you doing")

3

In [9]:
def bigram_jaccard(s1, s2):
    s1_tokenize = s1.split()
    s2_tokenize = s2.split()
    bigram_overlap = 0
    s1_bigrams = set([(s1_tokenize[i], s1_tokenize[i+1]) for i in range(0, len(s1_tokenize)-1)])
    s2_bigrams = set([(s2_tokenize[i], s2_tokenize[i+1]) for i in range(0, len(s2_tokenize)-1)])
    return float(len(s1_bigrams.intersection(s2_bigrams)))/float(len(s1_bigrams.union(s2_bigrams)))

bigram_jaccard("How are doing", "How are you doing")

0.25

In [10]:
def load_filecontents(filename):
    with open(filename) as f:
        file_content = f.readlines()
    return file_content

In [11]:
def build_features(filename):
    
    #doc2vec + cosine distance (TODO: get euclidean between vectors)
    pair = make_distance_vectors(filename)
    d2vresults = pair[0]
    file_contents = pair[1]
    features = []
    
    #Cosine distance between glove vector sentences
    cosresults = []
    for i in range(0, len(file_contents), 2):
        cos_dist = distance_between_sentences(file_contents[i], file_contents[i+1])
        cosresults.append(cos_dist)
    
    #word overlap (single word)
    wordoverlap = []
    for i in range(0, len(file_contents), 2):
        overlap = word_overlap(file_contents[i], file_contents[i+1])
        wordoverlap.append(overlap)
    
    #jaccard word overlap
    jaccards = []
    for i in range(0, len(file_contents), 2):
        j_dist = jaccard(file_contents[i], file_contents[i+1])
        jaccards.append(j_dist)
        
    #euclidean between golve vector sentences
    eucs = []
    for i in range(0, len(file_contents), 2):
        euc_dist = distance_between_sentences(file_contents[i], file_contents[i+1], distfunc = euclidean)
        eucs.append(euc_dist)
    
    #Question word indicator
    qword = []
    for i in range(0, len(file_contents), 2):
        q = qword_indicator(file_contents[i], file_contents[i+1])
        qword.append(q)
        
    #bigram overlap
    bigrams = []
    for i in range(0, len(file_contents), 2):
        b_overlap = bigram_overlap(file_contents[i], file_contents[i+1])
        bigrams.append(b_overlap)
        
    #bigram jaccard
    bigram_jaccard_arr = []
    for i in range(0, len(file_contents), 2):
        b_jaccard = bigram_jaccard(file_contents[i], file_contents[i+1])
        bigram_jaccard_arr.append(b_jaccard)
    
    #NER --> made below
    #Dependency Tree Features (Cosine dist of roots and diff in tree depth) --> made below
    
    features = [d2vresults, cosresults, wordoverlap, jaccards, eucs, qword, bigrams, bigram_jaccard_arr]
    return features

In [12]:
X_neg_all_features = build_features('new_neg_data_3.txt')
X_pos_all_features = build_features('new_pos_data_3.txt')
print 'examples vectorized'

examples vectorized


In [13]:
with open('stanford-corenlp-full-2015-12-09/negative_ner_raw.txt') as f:
    file_content = f.readlines()

NER = set(["LOCATION\n", "PERSON\n", "ORGANIZATION\n", "MONEY\n", "PERCENT\n", "DATE\n", "TIME\n"])
neg_named_entities = []
pos_named_entities = []

count = 0
for line in file_content:
    if line == '\n':
        neg_named_entities.append(count)
        count = 0
    elif line != '?\n':
        tokenized = line.split('\t')
        if tokenized[1] in NER:
            count += 1
    
f.close()

with open('stanford-corenlp-full-2015-12-09/positive_ner_raw.txt') as f2:
    file_content = f2.readlines()

count = 0
for line in file_content:
    if line == '\n':
        pos_named_entities.append(count)
        count = 0
    elif line != '?\n':
        tokenized = line.split('\t')
        if tokenized[1] in NER:
            count += 1
f2.close()

In [16]:
print len(pos_named_entities)
print len(X_pos_all_features[1])*2
print len(neg_named_entities)
print len(X_neg_all_features[1])*2

8492
8492
9448
9448


In [15]:
neg_named_entities.pop()
neg_named_entities.pop()
neg_named_entities.pop()

0

In [21]:
import pickle

ner_neg_pickled = open('ner_neg_pickled_2.txt', 'wb')
pickle.dump(neg_named_entities, ner_neg_pickled)
ner_neg_pickled.close()

ner_pos_pickled = open('ner_pos_pickled_2.txt', 'wb')
pickle.dump(pos_named_entities, ner_pos_pickled)
ner_pos_pickled.close()

In [15]:
import pickle
ner_neg_pickled = open('ner_neg_pickled.txt', 'rb')
neg_named_entities = pickle.load(ner_neg_pickled)
ner_neg_pickled.close()

ner_pos_pickled = open('ner_neg_pickled.txt', 'rb')
pos_named_entities = pickle.load(ner_pos_pickled)
ner_pos_pickled.close()

print len(neg_named_entities)
print len(pos_named_entities)

EOFError: 

In [17]:
# one contains named entity and other doesn't
ner_feature_indicator_neg = []
for i in range(0, len(neg_named_entities), 2):
    if neg_named_entities[i] == 0 and neg_named_entities[i+1]>0:
        ner_feature_indicator_neg.append(1)
    elif neg_named_entities[i+1] == 0 and neg_named_entities[i]>0:
        ner_feature_indicator_neg.append(1)
    else:
        ner_feature_indicator_neg.append(0)

ner_feature_indicator_pos = []
for i in range(0, len(pos_named_entities), 2):
    if pos_named_entities[i] == 0 and pos_named_entities[i+1]>0:
        ner_feature_indicator_pos.append(1)
    elif pos_named_entities[i+1] == 0 and pos_named_entities[i]>0:
        ner_feature_indicator_pos.append(1)
    else:
        ner_feature_indicator_pos.append(0)

In [18]:
X_neg_all_features.append(ner_feature_indicator_neg)
X_pos_all_features.append(ner_feature_indicator_pos)

In [61]:
X_features_master_neg = open('X_features_master_neg.txt', 'wb')
pickle.dump(X_neg_all_features, X_features_master_neg)
X_features_master_neg.close()

X_features_master_pos = open('X_features_master_pos.txt', 'wb')
pickle.dump(X_neg_all_features, X_features_master_pos)
X_features_master_pos.close()

In [62]:
A = open('X_features_master_neg.txt', 'rb')
test = pickle.load(A)

In [24]:
print len(X_neg_all_features[8])

4724


In [19]:
import itertools
subsets = []

feature_indices = [0,1,2,3,5,6,7,8]
for L in range(0, len(feature_indices)+1):
    for subset in itertools.combinations(feature_indices, L):
        subsets.append(subset)
        
subsets.pop(0)

()

In [20]:
y_pos = [1 for x in X_pos_all_features[0]]
y_neg = [0 for x in X_neg_all_features[0]]

print len(y_pos)*2
print len(y_neg)

8492
4724


In [158]:
X_pos

[[0.47593972196091539,
  0.055519788290799399,
  4,
  0.5714285714285714,
  5.207769374046325,
  0,
  2,
  0.2857142857142857,
  1],
 [0.55812436839118407,
  0.056775578995398868,
  5,
  0.45454545454545453,
  26.28430666731734,
  0,
  4,
  0.3333333333333333,
  0],
 [0.79972720617010495,
  0.1851849305769776,
  3,
  0.2727272727272727,
  28.590311390028837,
  0,
  0,
  0.0,
  1],
 [1.3350368965811135,
  0.035194860982142528,
  2,
  0.25,
  8.673662261774767,
  0,
  0,
  0.0,
  0],
 [0.36200039693730612,
  0.032435658299625469,
  5,
  0.5555555555555556,
  14.213654989702244,
  0,
  1,
  0.08333333333333333,
  0],
 [0.56853540615147269,
  0.15340313566103658,
  4,
  0.36363636363636365,
  18.625056546199886,
  1,
  2,
  0.18181818181818182,
  0],
 [1.4056427510592671,
  0.33317840318623215,
  2,
  0.16666666666666666,
  23.63077657716825,
  0,
  0,
  0.0,
  1],
 [0.3923407856684108,
  0.069576848459948604,
  4,
  0.4,
  9.530292303674187,
  0,
  1,
  0.09090909090909091,
  1],
 [0.8891

In [126]:
for t in test:
    print t

3
5
3
3
3
3
3
4
3
5
3
3
5
4
4
4
5
4
4
4
3
3
5
4
4
4
3
4
4
4
5
5
4
3
4
4
3
4
5
4
5
4
3
4
4
4
5
6
4
4
6
5
6
5
5
3
3
5
3
4
5
4
5
5
4
4
4
4
4
5
4
3
6
4
4
5
4
4
4
4
4
4
4
3
3
4
3
5
4
3
4
5
5
4
3
5
4
4
5
4
5
5
5
4
5
4
4
4
4
6
5
3
4
7
5
4
4
3
6
5
4
4
3
4
3
7
4
5
4
3
6
4
5
4
3
5
3
3
4
5
4
5
4
7
5
4
5
3
4
4
4
3
6
3
4
4
4
4
3
4
6
5
3
6
6
5
5
5
6
4
4
5
4
4
4
5
3
4
5
6
3
5
5
5
6
4
4
6
3
4
4
5
3
4
4
4
5
4
4
4
5
3
4
4
5
4
4
5
4
4
5
5
3
5
5
5
4
4
5
4
5
5
3
3
5
5
5
4
5
3
4
3
5
3
4
3
5
4
4
5
4
4
4
3
3
4
3
7
4
3
4
3
4
4
4
5
4
5
3
4
4
4
3
3
4
6
7
5
4
4
6
3
5
5
4
7
3
4
9
4
4
5
8
5
3
5
3
5
6
3
3
6
4
3
4
6
7
5
4
5
4
3
5
3
5
4
4
5
3
5
3
4
4
4
5
5
5
3
3
5
4
4
6
5
4
5
5
6
6
6
4
6
4
6
3
3
4
6
5
4
5
4
5
6
5
5
4
3
5
5
5
4
4
4
5
3
5
4
6
3
4
5
3
3
6
4
5
3
4
6
3
3
3
4
4
4
3
7
4
5
6
4
4
4
4
3
4
5
6
4
5
3
3
3
5
4
3
5
3
6
4
3
4
4
5
4
4
5
3
4
5
5
3
5
5
6
6
4
5
5
3
3
5
4
3
4
3
4
5
5
4
5
5
7
5
3
3
6
5
3
6
4
5
5
4
6
4
4
4
3
3
3
5
3
5
4
3
4
5
6
4
3
4
4
3
5
3
5
4
4
4
4
5
3
5
4
4
4
5
3
5
5
3
4
6
4
5
4
3
3
4
5
6
6
4
5
3
3
6
3


In [21]:
y = y_neg + y_pos
for s in subsets:
    indices = list(s)
    print "Testing features: " + str(indices).strip('[]')
    #build X

    X_neg = [[] for i in range(0, len(X_neg_all_features[0]))]
    X_pos = [[] for i in range(0, len(X_pos_all_features[0]))]
    for i in indices:
        neg_features = X_neg_all_features[i]
        j = 0
        for feature in neg_features:
            X_neg[j].append(feature)
            j += 1
        
        pos_features = X_pos_all_features[i]
        j = 0
        for feature in pos_features:
            X_pos[j].append(feature)
            j += 1
    
    X = X_neg + X_pos
    
    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    #train test accuracy
    clf = train(X_train, y_train, 1000, 2)
    predictions = test(clf, X_test)
    print('Accuracy: %0.03f' % accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions, digits=3))

Testing features: 0
finished training svm
finished predicting
Accuracy: 0.551
             precision    recall  f1-score   support

          0      0.546     0.874     0.672      1560
          1      0.576     0.191     0.287      1401

avg / total      0.560     0.551     0.490      2961

Testing features: 1
finished training svm
finished predicting
Accuracy: 0.654
             precision    recall  f1-score   support

          0      0.633     0.817     0.713      1560
          1      0.699     0.473     0.564      1401

avg / total      0.664     0.654     0.643      2961

Testing features: 2
finished training svm
finished predicting
Accuracy: 0.662
             precision    recall  f1-score   support

          0      0.613     0.969     0.751      1560
          1      0.902     0.320     0.473      1401

avg / total      0.750     0.662     0.620      2961

Testing features: 3
finished training svm
finished predicting
Accuracy: 0.662
             precision    recall  f1-score 

In [27]:
#import relevant dependency trees

import pickle
f1 = open('tree_roots_neg.txt', 'rb')
    # Pickle the 'data' dictionary using the highest protocol available.
neg_roots = pickle.load(f1)
f1.close()

f2 = open('tree_depth_neg.txt', 'rb')
    # Pickle the 'data' dictionary using the highest protocol available.
neg_depth = pickle.load(f2)
f2.close()

f3 = open('tree_roots_pos.txt', 'rb')
    # Pickle the 'data' dictionary using the highest protocol available.
pos_roots = pickle.load(f3)
f3.close()

f4 = open('tree_depth_pos.txt', 'rb')
    # Pickle the 'data' dictionary using the highest protocol available.
pos_depth = pickle.load(f4)
f4.close()

In [38]:
new_neg_roots = []
for root in neg_roots:
    new_root = root.encode('ascii','ignore')
    new_neg_roots.append(new_root)

new_pos_roots = []
for root in pos_roots:
    new_root = root.encode('ascii','ignore')
    new_pos_roots.append(new_root)

In [39]:
neg_roots_features = []
for i in range(0, len(neg_roots), 2):
    root_dist = distance_between_words(new_neg_roots[i], new_neg_roots[i+1])
    neg_roots_features.append(root_dist)
    
pos_roots_features = []
for i in range(0, len(pos_roots), 2):
    root_dist = distance_between_words(new_pos_roots[i], new_pos_roots[i+1])
    pos_roots_features.append(root_dist)
    
neg_depth_features = []
for i in range(0, len(neg_depth), 2):
    depth_diff = abs(neg_depth[i]-neg_depth[i+1])
    neg_depth_features.append(depth_diff)

pos_depth_features = []
for i in range(0, len(pos_depth), 2):
    depth_diff = abs(pos_depth[i]-pos_depth[i+1])
    pos_depth_features.append(depth_diff)

In [40]:
X_neg_all_features.append(neg_roots_features)
X_pos_all_features.append(pos_roots_features)
X_neg_all_features.append(neg_depth_features)
X_pos_all_features.append(pos_depth_features)

print len(X_neg_all_features)
print len(X_pos_all_features)

11
11


In [47]:
subsets2 = []

feature_indices2 = [0,1,2,3,5,6,7,8,9,10]
for L in range(0, len(feature_indices)+1):
    for subset in itertools.combinations(feature_indices2, L):
        subsets2.append(subset)

new_indices = []
for subset in subsets2:
    if subset not in subsets:
        new_indices.append(subset)

        
new_indices.pop(0)

()

In [49]:
len(subsets2)

1013

In [50]:

for s2 in new_indices:
    indices = list(s2)
    print "Testing features: " + str(indices).strip('[]')
    #build X

    X_neg = [[] for i in range(0, len(X_neg_all_features[0]))]
    X_pos = [[] for i in range(0, len(X_pos_all_features[0]))]
    for i in indices:
        neg_features = X_neg_all_features[i]
        j = 0
        for feature in neg_features:
            X_neg[j].append(feature)
            j += 1
        
        pos_features = X_pos_all_features[i]
        j = 0
        for feature in pos_features:
            X_pos[j].append(feature)
            j += 1
    
    X = X_neg + X_pos
    
    #train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    #train test accuracy
    clf = train(X_train, y_train, 1000, 2)
    predictions = test(clf, X_test)
    print('Accuracy: %0.03f' % accuracy_score(y_test, predictions))
    print(classification_report(y_test, predictions, digits=3))

Testing features: 9
finished training svm
finished predicting
Accuracy: 0.561
             precision    recall  f1-score   support

          0      0.569     0.684     0.621      1560
          1      0.546     0.424     0.477      1401

avg / total      0.559     0.561     0.553      2961

Testing features: 10
finished training svm
finished predicting
Accuracy: 0.534
             precision    recall  f1-score   support

          0      0.558     0.558     0.558      1560
          1      0.508     0.507     0.507      1401

avg / total      0.534     0.534     0.534      2961

Testing features: 0, 9
finished training svm
finished predicting
Accuracy: 0.571
             precision    recall  f1-score   support

          0      0.587     0.631     0.608      1560
          1      0.551     0.505     0.527      1401

avg / total      0.570     0.571     0.570      2961

Testing features: 0, 10
finished training svm
finished predicting
Accuracy: 0.572
             precision    recall  f